In [ ]:
#The goal of this TP is to propose a CNN for object classification
#We will use the newdata dataset during this TP
#Note : You should submit the completed notebook file and
#a pdf file describing the different results

In [ ]:
import os
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
%%capture
!pip install wandb -q


In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: otmattou (use `wandb login --relogin` to force relogin)


True

In [ ]:
%%capture
!unzip Newdata.zip && mv Newdata/Test/Shape1/ Newdata/Test/Cow/ && mv Newdata/Test/Shape2/ Newdata/Test/Horse/

In [ ]:
BATCH_SIZE = 16

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = ImageFolder('/content/Newdata/Train',transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)
testset = ImageFolder('/content/Newdata/Test',transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)
CLASS_NAMES = ('Cow', 'Horse')

In [ ]:
def show_batch(image_batch, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(15):
      ax = plt.subplot(5,5,n+1)
      img = image_batch[n] / 2 + 0.5     # unnormalize
      img = img.numpy()
      plt.imshow(np.transpose(img, (1, 2, 0)))
      plt.title(CLASS_NAMES[label_batch[n]])
      plt.axis('off')

In [ ]:
sample_images, sample_labels = next(iter(trainloader))
show_batch(sample_images, sample_labels)

In [ ]:
img = sample_images[1] / 2 + 0.5     # unnormalize
img = img.numpy()
plt.imshow(np.transpose(img, (1, 2, 0)))
plt.title(CLASS_NAMES[sample_labels[1]])
print(img.shape)

In [ ]:
#From scratch design a shallow CNN composed of a set of conv, pool and FC layers

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Activation, Dropout

# Je choisis d'implémenter l'architecture AlexNet du fait de sa simplicité

model = keras.models.Sequential([
    tf.keras.layers.Reshape(target_shape=(256,256,3),input_shape=(3,256,256)),#reshape our input to (256,256,3)
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(256,256,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='sigmoid') # problem with 2 classes
])

In [ ]:
model.build()
model.summary(line_length=None, positions=None, print_fn=None)

In [ ]:
#Train the model
#Plot the error vs epoch curve
#Report the accuracy on the test set

In [ ]:
model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
sample_images, sample_labels = next(iter(trainloader))
sample_images = sample_images.numpy()
sample_labels = sample_labels.numpy()
print(type(sample_images[1]))
print(type(sample_labels[1]))
print(sample_images[1].shape)
print(sample_labels[1].shape)
model.fit(sample_images,sample_labels, epochs=5)

In [ ]:
sample_images_test, sample_labels_test = next(iter(testloader))
sample_images_test = sample_images_test.numpy()
sample_labels_test = sample_labels_test.numpy()
model.evaluate(
    x=sample_images_test, y=sample_labels_test, batch_size=28, verbose=1, sample_weight=None, steps=None,
     max_queue_size=10, workers=1, use_multiprocessing=False
)

In [ ]:
model.evaluate(
    x=sample_images_test, y=sample_labels_test, batch_size=28, verbose=1, sample_weight=None, steps=None,
     max_queue_size=10, workers=1, use_multiprocessing=False)

In [ ]:
### YOUR CODE HERE


In [ ]:
#Load in pre-trained weights from a network trained on a large dataset

In [ ]:
from keras.applications.vgg16 import VGG16

model = VGG16() # Création du modèle VGG-16 implementé par Keras

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

sample_images, sample_labels = next(iter(trainloader))
sample_images = sample_images.numpy()
sample_labels = sample_labels.numpy()

sample_images = np.reshape(sample_images[0],(3,256,256))
data_tf = tf.convert_to_tensor(sample_images, np.float32)
print(np.shape(data_tf))
#img = preprocess_input(sample_images)  # Prétraiter l'image comme le veut VGG-16



In [ ]:
y = model.predict(data_tf)  # Prédir la classe de l'image (parmi les 1000 classes d'ImageNet)

In [ ]:
#Extract features from the train and test images using the pre-trained model

In [ ]:
#Classify the test images using a 1-nearst neighbour classifier

In [ ]:
#Load in pre-trained weights from a network trained on a large dataset
#Freeze all the weights in the convolutional layers

In [ ]:
#Replace the FC layers of the network with a custom classifier

In [ ]:
#Train only the FC layers for the object classification

In [ ]:
#Try different optimizers for the training